**Projet 2 : Explorer et comprendre les performances des employés**  

**Problématique métier:**  
Une entreprise souhaite analyser la répartition des performances des employés pour comprendre les écarts et identifier les outliers.  
  
  
**Objectif:**  
Étudier les distributions des scores de performance et des heures travaillées pour détecter les facteurs d'amélioration.

**Etapes:** 
1/Exploration du fichier et nettoyage des données.   
2/Statistiques descriptives : Moyenne, médiane, mode, quartiles, variance, écart-type.  
3/Visualisation des distributions : Histogrammes, boxplots, pie chart, diagramme en barre.  
4/Détection et analyse des outliers avec la règle des 1.5 * IQR.  


In [89]:
import pandas as pd

path='D:\\Documents\\VSCode\\Projet HR sur python\\HRDataset.csv'
# je crée un raw_data et je crée une copie que je vais ensuite nettoyer
raw_data=pd.read_csv(path)
data=raw_data.copy()

1/Exploration du fichier et nettoyage des données. 

In [90]:
data.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,...,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,...,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,...,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,...,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,...,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2


On peut déduire que le genre ID correspond à 1 pour homme et 0 pour femme.  
Score ID semble être un score allant de 1 à 4.  
Les colonnes avec le noms ne sont pas importantes pour cette étude puisqu'ils on un ID.
Les colonnes comme MarriedID, MaritalStatusID ne semble pas pertinentes à cette études.  
La colonnes empStatus est pas claire donc à voir si on la garde sur le long terme.     

In [91]:

#se débarrasser des colonnes inutiles et contenant des données sensibles
try:
    data.pop('MarriedID')
    data.pop('MaritalStatusID')
    data.pop('Employee_Name')
    data.pop('ManagerName')
except KeyError:
    pass 

data.head()


,EmpID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,...,Department,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,10026,1,1,5,4,0,62506,0,19,Production Technician I,...,Production,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,10084,1,5,3,3,0,104437,1,27,Sr. DBA,...,IT/IS,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,10196,0,5,5,3,0,64955,1,20,Production Technician II,...,Production,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,10088,0,1,5,3,0,64991,0,19,Production Technician I,...,Production,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,10069,0,5,5,3,0,50825,1,19,Production Technician I,...,Production,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2


Etude de la structure du dataFrame

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   EmpID                       311 non-null    int64  
 1   GenderID                    311 non-null    int64  
 2   EmpStatusID                 311 non-null    int64  
 3   DeptID                      311 non-null    int64  
 4   PerfScoreID                 311 non-null    int64  
 5   FromDiversityJobFairID      311 non-null    int64  
 6   Salary                      311 non-null    int64  
 7   Termd                       311 non-null    int64  
 8   PositionID                  311 non-null    int64  
 9   Position                    311 non-null    object 
 10  State                       311 non-null    object 
 11  Zip                         311 non-null    int64  
 12  DOB                         311 non-null    object 
 13  Sex                         311 non

Il nous reste encore 31 collonnes, je vais vérifier la pertinence des colonnes:  
13  Sex                         311 non-null    object   
14  MaritalDesc                 311 non-null    object   
15  CitizenDesc                 311 non-null    object   
16  HispanicLatino              311 non-null    object   
17  RaceDesc                    311 non-null    object   
Les dates 'DateofHire' et 'DateofTermination'  devront être converties en date. 
Il semble aussi qu'il n'y ait pas de valeurs nulles.   

In [93]:
data.isna().sum() #confirme l'absence de valeur manquantes 

EmpID                           0
GenderID                        0
EmpStatusID                     0
DeptID                          0
PerfScoreID                     0
FromDiversityJobFairID          0
Salary                          0
Termd                           0
PositionID                      0
Position                        0
State                           0
Zip                             0
DOB                             0
Sex                             0
MaritalDesc                     0
CitizenDesc                     0
HispanicLatino                  0
RaceDesc                        0
DateofHire                      0
DateofTermination             207
TermReason                      0
EmploymentStatus                0
Department                      0
ManagerID                       8
RecruitmentSource               0
PerformanceScore                0
EngagementSurvey                0
EmpSatisfaction                 0
SpecialProjectsCount            0
LastPerformanc

In [94]:
import pyarrow as pa

# Conversion des dates
try:
    data['LastPerformanceReview_Date'] = data['LastPerformanceReview_Date'].astype('date32[pyarrow]')
    data['DateofHire'] = data['DateofHire'].astype('date32[pyarrow]')
    data['DateofTermination'] = data['DateofTermination'].astype('date32[pyarrow]')
    data['DOB'] = pd.to_datetime(data['DOB'], format='%m/%d/%y').astype('date32[pyarrow]')



except Exception as e:
    print(f'Erreur : {e}')

#affichage des colonnes à analyser    
colonnes13_17 = data.iloc[:10, 13:18] 
colonnes13_17



,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc
0,M,Single,US Citizen,No,White
1,M,Married,US Citizen,No,White
2,F,Married,US Citizen,No,White
3,F,Married,US Citizen,No,White
4,F,Divorced,US Citizen,No,White
5,F,Single,US Citizen,No,White
6,F,Single,US Citizen,No,White
7,M,Widowed,US Citizen,No,White
8,F,Single,US Citizen,No,Black or African American
9,M,Divorced,US Citizen,No,White


In [95]:
for colonne in colonnes13_17:
    print(f'{colonne}: {data[colonne].unique()}')

Sex: ['M ' 'F']
MaritalDesc: ['Single' 'Married' 'Divorced' 'Widowed' 'Separated']
CitizenDesc: ['US Citizen' 'Eligible NonCitizen' 'Non-Citizen']
HispanicLatino: ['No' 'Yes' 'no' 'yes']
RaceDesc: ['White' 'Black or African American' 'Two or more races' 'Asian'
 'American Indian or Alaska Native' 'Hispanic']


Les colonnes sont soit redondantes soit pas pertinentes dans la mesure de la performance d'une équipe. Je vais donc les exclure de l'étude.

In [96]:
for colonne in colonnes13_17:
    data.pop(colonne)

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   EmpStatusID                 311 non-null    int64               
 3   DeptID                      311 non-null    int64               
 4   PerfScoreID                 311 non-null    int64               
 5   FromDiversityJobFairID      311 non-null    int64               
 6   Salary                      311 non-null    int64               
 7   Termd                       311 non-null    int64               
 8   PositionID                  311 non-null    int64               
 9   Position                    311 non-null    object              
 10  State                       311 non-null    object

In [98]:
data.pop('DeptID')



0      5
1      3
2      5
3      5
4      5
      ..
306    5
307    5
308    3
309    3
310    5
Name: DeptID, Length: 311, dtype: int64

In [99]:
data['FromDiversityJobFairID'].unique()
#je ne comprends pas ce que ça signifie donc je supprime
data.pop('FromDiversityJobFairID')


0      0
1      0
2      0
3      0
4      0
      ..
306    0
307    0
308    0
309    0
310    0
Name: FromDiversityJobFairID, Length: 311, dtype: int64

In [100]:
data['Termd'].unique()
#je ne comprends pas ce que ça signifie don je supprime
data.pop('Termd')


0      0
1      1
2      1
3      0
4      1
      ..
306    0
307    1
308    0
309    0
310    0
Name: Termd, Length: 311, dtype: int64

In [101]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   EmpStatusID                 311 non-null    int64               
 3   PerfScoreID                 311 non-null    int64               
 4   Salary                      311 non-null    int64               
 5   PositionID                  311 non-null    int64               
 6   Position                    311 non-null    object              
 7   State                       311 non-null    object              
 8   Zip                         311 non-null    int64               
 9   DOB                         311 non-null    date32[day][pyarrow]
 10  DateofHire                  311 non-null    date32

In [102]:
data.pop('EmpStatusID') #en doublon avec EmpStatus



0      1
1      5
2      5
3      1
4      5
      ..
306    1
307    5
308    1
309    1
310    1
Name: EmpStatusID, Length: 311, dtype: int64

In [103]:
len(data['Position'].unique())


32

In [104]:
len(data['PositionID'].unique())

30

En comparant les longueurs des colonnes 'Position' et 'PositionID' j'ai remarqué que PositionID n'a que 32 valeurs possibles alors que Position en a 32. ceci peut être lié à un problème de formatage que je vais essayer de trouver.

In [105]:

# Créer une liste pour stocker les paires
pairs = []

# Itérer sur chaque ligne du DataFrame
for row in data.itertuples():
    # Ajouter la paire à la liste
    pairs.append((row.Position, row.PositionID))
# Créer un tuple que avec les valeurs uniques avec un set
pairs=set(pairs)
pairs=list(pairs)

pairs=sorted(pairs, key= lambda tup: tup[1])

for tup in pairs:
    print(tup)
    




('Accountant I', 1)
('Administrative Assistant', 2)
('Area Sales Manager', 3)
('BI Developer', 4)
('BI Director', 5)
('CIO', 6)
('Data Architect', 7)
('Database Administrator', 8)
('Data Analyst ', 9)
('Data Analyst', 9)
('Director of Operations', 10)
('Director of Sales', 11)
('IT Director', 12)
('IT Manager - DB', 13)
('IT Manager - Support', 13)
('IT Manager - Infra', 13)
('IT Support', 14)
('Network Engineer', 15)
('President & CEO', 16)
('Production Manager', 17)
('Production Manager', 18)
('Production Technician I', 19)
('Production Technician II', 20)
('Sales Manager', 21)
('Senior BI Developer', 22)
('Software Engineer', 23)
('Shared Services Manager', 23)
('Software Engineer', 24)
('Software Engineering Manager', 25)
('Sr. Accountant', 26)
('Sr. DBA', 27)
('Sr. Network Engineer', 28)
('Principal Data Architect', 29)
('Enterprise Architect', 30)


En triant on se rend compte l'id 13 sert à 3 postes de manager IT différents :  
('IT Manager - DB', 13)  
('IT Manager - Infra', 13)  
('IT Manager - Support', 13)  

Et on a un problème de espaces :  
('Data Analyst', 9)  
('Data Analyst ', 9)  

In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   PerfScoreID                 311 non-null    int64               
 3   Salary                      311 non-null    int64               
 4   PositionID                  311 non-null    int64               
 5   Position                    311 non-null    object              
 6   State                       311 non-null    object              
 7   Zip                         311 non-null    int64               
 8   DOB                         311 non-null    date32[day][pyarrow]
 9   DateofHire                  311 non-null    date32[day][pyarrow]
 10  DateofTermination           104 non-null    date32

In [107]:
# Initialiser un compteur
compteur = 0  

# Fonction pour nettoyer et compter les colonnes traitées
def nettoyer_colonne(col):
    global compteur
    if col.dtype == 'object':  # Vérifier si la colonne est de type objet
        compteur += 1  # Incrémenter pour chaque colonne traitée
        return col.map(lambda x: x.strip() if isinstance(x, str) else x)
    return col

# Copier les données initiales pour comparaison
data_before = data.copy()

# Appliquer la fonction à chaque colonne
data = data.apply(nettoyer_colonne)

# Identifier les colonnes où des modifications ont eu lieu
colonnes_modifiées = [col for col in data.columns if not data[col].equals(data_before[col])]

# Afficher les résultats
print(f"Nombre de colonnes traitées : {compteur}")
print(f"Colonnes de type objet : {len(data.select_dtypes(include='object').columns)}")
print(f"Colonnes modifiées : {colonnes_modifiées}")


Nombre de colonnes traitées : 7
Colonnes de type objet : 7
Colonnes modifiées : ['Position', 'Department']


Je vais utiliser la même dmarche cette fois ci pour supprimer les doubles espaces et passer tout en miniscule pour faciliter le traitement.

In [108]:
# Initialiser un compteur
compteur = 0  

# Fonction pour nettoyer et compter les colonnes traitées
def suppression_double_espaces(col):
    global compteur
    if col.dtype == 'object':  # Vérifier si la colonne est de type objet
        compteur += 1  # Incrémenter pour chaque colonne traitée
        return col.map(lambda x: x.replace("  "," ") if isinstance(x, str) else x)
    return col

# Copier les données initiales pour comparaison
data_before = data.copy()

# Appliquer la fonction à chaque colonne
data = data.apply(suppression_double_espaces)

# Identifier les colonnes où des modifications ont eu lieu
colonnes_modifiées = [col for col in data.columns if not data[col].equals(data_before[col])]

# Afficher les résultats
print(f"Nombre de colonnes traitées : {compteur}")
print(f"Colonnes de type objet : {len(data.select_dtypes(include='object').columns)}")
print(f"Colonnes modifiées : {colonnes_modifiées}")

Nombre de colonnes traitées : 7
Colonnes de type objet : 7
Colonnes modifiées : []


In [109]:
compteur2=0
def suppression_double_espaces(col):
    global compteur2
    if col.dtype == 'object':  # Vérifie que la colonne est de type objet
        compteur2 += 1
        return col.map(lambda x: x.replace("  ", " ") if isinstance(x, str) else x) 
    return col  # Retourne la colonne telle quelle si ce n'est pas un type 'object'

compteur3=0
def minimiser_valeurs(col):
    global compteur3
    if col.dtype=='object':
        compteur3+=1
        return col.map(lambda x : x.lower() if isinstance(x,str) else x)
    return col


In [110]:
# Copier les données initiales pour comparaison
data_before = data.copy()
data=data.apply(suppression_double_espaces)
data=data.apply(minimiser_valeurs)
# Identifier les colonnes où des modifications ont eu lieu
colonnes_modifiées = [col for col in data.columns if not data[col].equals(data_before[col])]
print(compteur2, compteur3)
print(colonnes_modifiées)

7 7
['Position', 'State', 'TermReason', 'EmploymentStatus', 'Department', 'RecruitmentSource', 'PerformanceScore']


Je vais continuer à analyser les valeurs uniques de colonnes objet pour assayer de détecter d'autres erreur de formatage.

In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   PerfScoreID                 311 non-null    int64               
 3   Salary                      311 non-null    int64               
 4   PositionID                  311 non-null    int64               
 5   Position                    311 non-null    object              
 6   State                       311 non-null    object              
 7   Zip                         311 non-null    int64               
 8   DOB                         311 non-null    date32[day][pyarrow]
 9   DateofHire                  311 non-null    date32[day][pyarrow]
 10  DateofTermination           104 non-null    date32

In [112]:
data['TermReason'].unique()

array(['n/a-stillemployed', 'career change', 'hours', 'return to school',
       'another position', 'unhappy', 'attendance', 'performance',
       'learned that he is a gangster', 'retiring',
       'relocation out of area', 'more money', 'military',
       'no-call, no-show', 'fatal attraction',
       'maternity leave - did not return', 'medical issues',
       'gross misconduct'], dtype=object)

Peut être que 'learned that he is a gangster' et 'fatal attraction' peuvent rentrer dans la catégorie 'gross misconduct'

In [113]:
data.loc[(data['TermReason']=='learned that he is a gangster')|(data['TermReason']=='fatal attraction'),['TermReason']]='gross misconduct'
data.loc[data['TermReason']=='gross misconduct', 'TermReason']

25     gross misconduct
95     gross misconduct
293    gross misconduct
Name: TermReason, dtype: object

Traitement des valeurs nulles dans 'Manager ID'

In [114]:
data['ManagerID'].unique()
data.loc[data['ManagerID'].isna(), 'ManagerID']

19    NaN
30    NaN
44    NaN
88    NaN
135   NaN
177   NaN
232   NaN
251   NaN
Name: ManagerID, dtype: float64

In [115]:
#recherche des noms des managers dans raw_data
raw_data.loc[raw_data['ManagerID'].isna(), ['ManagerName', 'ManagerID']]



,ManagerName,ManagerID
19,Webster Butler,NaN
30,Webster Butler,NaN
44,Webster Butler,NaN
88,Webster Butler,NaN
135,Webster Butler,NaN
177,Webster Butler,NaN
232,Webster Butler,NaN
251,Webster Butler,NaN


C'est toujours le même manager qui manque, je vais donc chercher si son ID apparaît dans d'autres lignes

In [116]:
raw_data.loc[raw_data['ManagerName']=='Webster Butler', ['ManagerName', 'ManagerID']]

,ManagerName,ManagerID
4,Webster Butler,39.0
19,Webster Butler,NaN
30,Webster Butler,NaN
44,Webster Butler,NaN
65,Webster Butler,39.0
88,Webster Butler,NaN
89,Webster Butler,39.0
105,Webster Butler,39.0
124,Webster Butler,39.0
135,Webster Butler,NaN


L'ID de 'Webster Butler' est 39 donc je vais remplacer les valeurs nulles par 39

In [117]:
data.loc[pd.isna(data['ManagerID']), 'ManagerID'] = 39
data.iloc[276, 14]# contrôle que la modif a bien eu lieu correctement


np.float64(39.0)

In [118]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   PerfScoreID                 311 non-null    int64               
 3   Salary                      311 non-null    int64               
 4   PositionID                  311 non-null    int64               
 5   Position                    311 non-null    object              
 6   State                       311 non-null    object              
 7   Zip                         311 non-null    int64               
 8   DOB                         311 non-null    date32[day][pyarrow]
 9   DateofHire                  311 non-null    date32[day][pyarrow]
 10  DateofTermination           104 non-null    date32

In [119]:
#verifie la cohérence entre PerformanceScore et PerfScoreID
combinaisons_possibles = set(data.loc[(data['PerformanceScore'].notna()) | (data['PerfScoreID'].notna()), ['PerformanceScore', 'PerfScoreID']].itertuples(index=False, name=None))

for valeur in combinaisons_possibles:
    print(valeur)

('fully meets', 3)
('pip', 1)
('exceeds', 4)
('pip', 3)
('needs improvement', 2)
('fully meets', 1)


il y a une erreur entre ('fully meets', 1) je vais donc aligner le PerfScoreID avec un score de 3

In [120]:
data.loc[(data['PerfScoreID']==1) & (data['PerformanceScore']=='fully meets'), 'PerfScoreID']=3
data.iloc[64,:]#ligne 64 parce que j'ai vérifié avant que c'était là l'erreur
data.pop('PerformanceScore')#je supprime pour éviter de doubles l'info avec ScoreID

0          exceeds
1      fully meets
2      fully meets
3      fully meets
4      fully meets
          ...     
306    fully meets
307            pip
308        exceeds
309    fully meets
310    fully meets
Name: PerformanceScore, Length: 311, dtype: object

In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype               
---  ------                      --------------  -----               
 0   EmpID                       311 non-null    int64               
 1   GenderID                    311 non-null    int64               
 2   PerfScoreID                 311 non-null    int64               
 3   Salary                      311 non-null    int64               
 4   PositionID                  311 non-null    int64               
 5   Position                    311 non-null    object              
 6   State                       311 non-null    object              
 7   Zip                         311 non-null    int64               
 8   DOB                         311 non-null    date32[day][pyarrow]
 9   DateofHire                  311 non-null    date32[day][pyarrow]
 10  DateofTermination           104 non-null    date32

In [122]:
#verifie la cohérence entre GenreID et Genre
combinaisons_possibles = set(raw_data.loc[(raw_data['GenderID'].notna()) | (raw_data['Sex'].notna()), ['GenderID', 'Sex']].itertuples(index=False, name=None))

for valeur in combinaisons_possibles:
    print(valeur)

(0, 'F')
(1, 'M ')


Recherche cohérence entre DeptID et Department

In [123]:
combinaisons_possibles = set(raw_data.loc[(raw_data['DeptID'].notna()) | (raw_data['Department'].notna()), ['DeptID', 'Department']].itertuples(index=False, name=None))
combinaisons_possibles=sorted(combinaisons_possibles, key=lambda tup : tup[1])
for valeur in combinaisons_possibles:
    print(valeur)


(1, 'Admin Offices')
(2, 'Executive Office')
(3, 'IT/IS')
(5, 'Production       ')
(6, 'Production       ')
(6, 'Sales')
(1, 'Software Engineering')
(4, 'Software Engineering')


In [124]:
#correction du production avec un ID de 6 et je le passe à 5
raw_data.loc[(raw_data['DeptID']==6) & (raw_data['Department']=='Production       '), ['DeptID', 'Department']]
#l'erreur est ligne 64 mais pas besoin de corriger sur notre data puiqu'on a supprimé la colonne qui contient l'erreur

,DeptID,Department
64,6,Production


Recherche de outliers dans mes données numériques 

In [125]:
data.describe()

,EmpID,GenderID,PerfScoreID,Salary,PositionID,Zip,DOB,DateofHire,DateofTermination,ManagerID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
count,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311,311,104,311.000000,311.000000,311.000000,311.000000,311,311.000000,311.000000
mean,10156.000000,0.434084,2.983923,69020.684887,16.845659,6555.482315,1992-08-09,2013-02-03,2015-06-14,15.199357,4.110000,3.890675,1.218650,2017-09-10,0.414791,10.237942
min,10001.000000,0.000000,1.000000,45046.000000,1.000000,1013.000000,1969-02-09,2006-01-09,2010-08-30,1.000000,1.120000,1.000000,0.000000,2010-07-14,0.000000,1.000000
25%,10078.500000,0.000000,3.000000,55501.500000,18.000000,1901.500000,1978-01-24,2011-07-11,2014-01-10,10.000000,3.690000,3.000000,0.000000,2016-03-19,0.000000,5.000000
50%,10156.000000,0.000000,3.000000,62810.000000,19.000000,2132.000000,1983-12-02,2013-02-18,2015-09-22,16.000000,4.280000,4.000000,0.000000,2019-01-16,0.000000,10.000000
75%,10233.500000,1.000000,3.000000,72036.000000,20.000000,2355.000000,1988-06-17,2014-09-23,2016-09-08,19.500000,4.700000,5.000000,0.000000,2019-02-04,0.000000,15.000000
max,10311.000000,1.000000,4.000000,250000.000000,30.000000,98052.000000,2068-10-10,2018-07-09,2018-11-10,39.000000,5.000000,5.000000,8.000000,2019-02-28,6.000000,20.000000
std,89.922189,0.496435,0.576193,25156.636930,6.223419,16908.396884,NaN,NaN,NaN,8.864507,0.789938,0.909241,2.349421,NaN,1.294519,5.852596


J'ai trouvé un employé née en 2068 ce qui bien sur une valeur aberrante. Je vais le remplacer par 1968 qui est plus cohérant

In [126]:
data['DOB'] = data['DOB'].astype('datetime64[ns]')
# Appliquer le décalage de 100 ans uniquement sur les lignes filtrées de 'data'
data.loc[data['DOB'] > '2007-01-01', 'DOB'] = data.loc[data['DOB'] > '2007-01-01', 'DOB'] - pd.DateOffset(years=100)

# Vérifier les résultats
print(data.loc[data['DOB'] > '2007-01-01', 'DOB'])



Series([], Name: DOB, dtype: datetime64[ns])


Il ne reste plus qu'à chercher les doublons 

In [127]:
colonnes1_5 = data.columns[1:6]
doublons = data[data.duplicated(subset=colonnes1_5, keep=False)]
doublons


,EmpID,GenderID,PerfScoreID,Salary,PositionID,Position,State,Zip,DOB,DateofHire,...,EmploymentStatus,Department,ManagerID,RecruitmentSource,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences


J'ai pas de doublons donc j'ai terminé le nettoyages des donnés.

**Récapitulatif du nettoyage :**  
Colonnes supprimées : 
*MarriedID*  
*MaritalStatusID*  
*Employee_Name*  
*ManagerName*  
*Sex*   
*MaritalDesc*  
*CitizenDesc*          
*HispanicLatino*             
*RaceDesc*  

Parce que dans le cadre de l'entreprise l'évaluation d'un employé en fonction de son sexe, race ou statut marital, va l'encontre de la législation.  
  
*EmpStatusID :*en doublon avec EmpStatus  
*Termd:* donnée pas explicite  
*FromDiversityJobFairID:* donnée pas explicite  
*DeptID:* en doublon avec Dept  
*PerformanceScore*: en doublon avec Performance
  
Colonnes converties en date :  
    LastPerformanceReview_Date  
    DateofHire  
    DateofTermination  
    DOB  

Rercherche de valeurs vides effectuée: 8 à managerID retrouvés en comparant managerID et managerName
  
Formatage des colonnes objet pour supprimer les espaces inutiles et tout passer en miniscule pour faciliter les traitement du data.

Recherche de doublons que j'ai pas trouvé.





In [128]:
data.head()

,EmpID,GenderID,PerfScoreID,Salary,PositionID,Position,State,Zip,DOB,DateofHire,...,EmploymentStatus,Department,ManagerID,RecruitmentSource,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,10026,1,4,62506,19,production technician i,ma,1960,1983-07-10,2011-07-05,...,active,production,22.0,linkedin,4.60,5,0,2019-01-17,0,1
1,10084,1,3,104437,27,sr. dba,ma,2148,1975-05-05,2015-03-30,...,voluntarily terminated,it/is,4.0,indeed,4.96,3,6,2016-02-24,0,17
2,10196,0,3,64955,20,production technician ii,ma,1810,1988-09-19,2011-07-05,...,voluntarily terminated,production,20.0,linkedin,3.02,3,0,2012-05-15,0,3
3,10088,0,3,64991,19,production technician i,ma,1886,1988-09-27,2008-01-07,...,active,production,16.0,indeed,4.84,5,0,2019-01-03,0,15
4,10069,0,3,50825,19,production technician i,ma,2169,1989-09-08,2011-07-11,...,voluntarily terminated,production,39.0,google search,5.00,4,0,2016-02-01,0,2


In [129]:
data.to_pickle("data_cleaned.pkl") #enregistre le df nettoyé dans un fichier pickle
